In [1]:
import cogsworth
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# this all just makes plots look nice
%config InlineBackend.figure_format = 'retina'

plt.rc('font', family='serif')
plt.rcParams['text.usetex'] = False
fs = 24

# update various fontsizes to match
params = {'figure.figsize': (12, 8),
          'legend.fontsize': fs,
          'axes.labelsize': fs,
          'xtick.labelsize': 0.9 * fs,
          'ytick.labelsize': 0.9 * fs,
          'axes.linewidth': 1.1,
          'xtick.major.size': 7,
          'xtick.minor.size': 4,
          'ytick.major.size': 7,
          'ytick.minor.size': 4}
plt.rcParams.update(params)
pd.options.display.max_columns = 999

In [171]:
p_half = cogsworth.pop.Population(n_binaries=10000,
                                  sampling_params={"keep_singles": True, "m2_min": 0.01},
                                  BSE_settings={"binfrac": 0.5})
p_all = cogsworth.pop.Population(n_binaries=10000,
                                 sampling_params={"keep_singles": True, "m2_min": 0.01},
                                 BSE_settings={"binfrac": 1.0})

In [170]:
for p in [p_half, p_all]:
    p.sample_initial_binaries()
    n_binaries = (p._initial_binaries["kstar_2"] != 15).sum()
    f_binaries = n_binaries / len(p)
    print(f"Sampled binary fraction: {f_binaries:1.1f}")

Sampled binary fraction: 0.5
Sampled binary fraction: 1.0


In [270]:
p = cogsworth.pop.Population(n_binaries=100,
                             final_kstar1=[13, 14], final_kstar2=[13, 14],
                             BSE_settings={"binfrac": 1.0, "sigma": 30})
p.sample_initial_binaries()
p.perform_stellar_evolution()

First let's see what fraction of the population were disrupted when drawing kicks from a Maxwellian centred on $30 \, {\rm km / s}$

In [271]:
# count the number of binaries that got kicks while they hadn't already merged
n_kicked = len(p.bpp[((p.bpp["evol_type"] == 15) | (p.bpp["evol_type"] == 16)) & (p.bpp["sep"] != 0.0)]["bin_num"].unique())
f'{p.disrupted.sum() / n_kicked:1.2f}'

'0.42'

Now let's change the Maxwellian to be centred on $1000 \, {\rm km / s}$ and do the same for electron-capture supernovae.

In [272]:
p.BSE_settings["sigma"] = 1000
p.BSE_settings["sigmadiv"] = -1000
p.perform_stellar_evolution()

In [273]:
# count the number of binaries that got kicks while they hadn't already merged
n_kicked = len(p.bpp[((p.bpp["evol_type"] == 15) | (p.bpp["evol_type"] == 16)) & (p.bpp["sep"] != 0.0)]["bin_num"].unique())
f'{p.disrupted.sum() / n_kicked:1.2f}'

'0.92'

As you'd expect, the fraction is now much higher (and any that aren't disrupting are likely getting fallback adjusted kicks, got lucky directions, or had extremely high orbital velocities).